# Лабораторная Работа 1

### Перевозчиков Г. П.

In [79]:
from typing import List
from copy import deepcopy

class GaussSolver:
    def __init__(self, A: List[List[float]], B: List[float]) -> None:
        self.A = deepcopy(A)
        self.B = deepcopy(B)

    def SwapRows(self, A, B, row1, row2):
        """
        Замена строк местами
        """
        A[row1], A[row2] = A[row2], A[row1]
        B[row1], B[row2] = B[row2], B[row1]

    def DivideRow(self, A, B, row, divider):
        """
        Деление строки на число
        """
        A[row] = [a / divider for a in A[row]]
        B[row] /= divider

    def CombineRows(self, A, B, row, source_row, weight):
        """
        Сложение строки с другой строкой, умноженной на число
        """
        A[row] = [(a + k * weight) for a, k in zip(A[row], A[source_row])]
        B[row] += B[source_row] * weight
    
    def NormalizeSysten(self, A, B):
        """
        Приведение вистемы к треугольному виду
        """
        column = 0
        while (column < len(B)):
            current_row = None
            for r in range(column, len(A)):
                if current_row is None or abs(A[r][column]) > abs(A[current_row][column]):
                     current_row = r

            if current_row is None:
                raise Exception('Нет решений')
            if current_row != column:
                self.SwapRows(A, B, current_row, column)
                
            self.DivideRow(A, B, column, A[column][column])
            for r in range(column + 1, len(A)):
                self.CombineRows(A, B, r, column, -A[r][column])
            column += 1

    def Solve(self):
        """
        Решение системы методом гаусса
        """
        A = deepcopy(self.A)
        B = deepcopy(self.B)
        self.NormalizeSysten(A, B)
        X = [0 for b in B]
        for i in range(len(B) - 1, -1, -1):
            X[i] = B[i] - sum(x * a for x, a in zip(X[(i + 1):], A[i][(i + 1):]))
        return X

    def Det(self):
        """
        Нахождение определителя
        """
        det = 1
        A = deepcopy(self.A)
        B = deepcopy(self.B)
        self.NormalizeSysten(A, B)
        for i in range(len(self.B)):
            det = det * A[i][i]
        return det

    def Pivotize(self, mat_a, x):
        """
        Путем обмена строк расположить наибольшие элементы на диагонали
        """
        mat_a = deepcopy(mat_a)
        size = len(mat_a)
        row = max(range(x, size), key=lambda i: abs(mat_a[i][x]))
        if x != row:
            mat_a[x], mat_a[row] = mat_a[row], mat_a[x]
        return mat_a


    def Invert(self):
        """
        Обращение матрицы методом Гаусса-Жордана
        """
        mat_a = deepcopy(self.A)
        n = len(mat_a)

        # Дополнить матрицу справа единичной матрицей
        for i in range(n):
            mat_a[i] += [int(i == j) for j in range(n)]

        # Прямой ход
        for x in range(n):
            mat_a = self.Pivotize(mat_a, x)
            for i in range(x + 1, n):
                coefficient = mat_a[i][x] / mat_a[x][x]
                for j in range(x, n * 2):
                    mat_a[i][j] -= coefficient * mat_a[x][j]

        # Обратный ход
        for x in reversed(range(n)):
            for i in reversed(range(x)):
                coefficient = mat_a[i][x] / mat_a[x][x]
                for j in reversed(range(n * 2)):
                    mat_a[i][j] -= coefficient * mat_a[x][j]

        # Разделить строки на ведущие элементы
        for i in range(n):
            denominator = mat_a[i][i]
            for j in range(n * 2):
                mat_a[i][j] /= denominator

        # Оставить только правую часть матрицы
        for i in range(n):
            mat_a[i] = mat_a[i][n:]

        return mat_a

In [81]:
#1 11  Методом Гаусса вычислить решение СЛАУ,
#       определитель системы, обратную матрицу
#         -4  -6  -3  -4  -2           48 
#         -8 -12   4  -5  11          105 
#        -12 -18  -9  -3  11          183 
#        -11  -4   5  -2  -3          -83 
#        -11  -5   4   6  -4         -132 

myA=[
    [4, -6, -3, -4, -2],
    [-8, -12, 4, -5, 11],
    [-12, -18, -9, -3, 11],
    [-11, -4, 5, -2, -3],
    [-11,  -5, 4, 6, -4]
]
myB = [
    48,
    105,
    183,
    -83,
    -132
]
n = len(myA)

solver = GaussSolver(myA, myB)
X = solver.Solve()
print("Решение системы:")
print("\n".join("X{0} = {1:5.2f}".format(i + 1, x) for i, x in enumerate(X)))
print()

det = solver.Det()
print(f'Определитель:\n{det}')
print()

inverted = solver.Invert()
print(f'Обратная матрица:')
for i in range(n):
    for j in range(n):
        print('{0:5.2f}'.format(round(inverted[i][j], 10)), end=' ')
    print()

Решение системы:
X1 =  3.23
X2 = -1.62
X3 = -7.16
X4 = -5.97
X5 = 10.02

Определитель:
1.0

Обратная матрица:
 0.07  0.04 -0.04 -0.08  0.03 
-0.10 -0.04  0.01  0.06 -0.07 
 0.02  0.09 -0.07 -0.02  0.04 
-0.02  0.01 -0.01 -0.13  0.12 
-0.07  0.05  0.01 -0.05 -0.02 
